We installed the below dependencies via requirements.txt

In [15]:
#!pip install -qU \
#    nemoguardrails==0.4.0 \
#    openai==0.27.8

We need to set our OpenAI API key:

In [9]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../.env")

True

We can run Guardrails from terminal using the command:

```
nemoguardrails chat --config=config/
```

Where the `config` directory must contain our `config.yml` and a Colang file (like `topics.co`).

Alternatively we can load them from file using `RailsConfig.from_path("./config")` or from string variables in our code like so:

In [30]:
yaml_content = """
models:
- type: main
  engine: azure
  model: gpt-3.5
"""
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [27]:
import os
from nemoguardrails import LLMRails, RailsConfig

# https://github.com/NVIDIA/NeMo-Guardrails/issues/72#issuecomment-1645603818
from langchain_openai.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(deployment_name=os.getenv("AZURE_LLM_MODEL_DEPLOYMENT_NAME"))


# initialize rails config
config = RailsConfig.from_content(
  	#yaml_content=yaml_content, # We pass the LLM directly to the LLMRails
    colang_content=colang_content
)
# create rails
rails = LLMRails(config, llm = llm)

From here, we begin asking questions and interacting with our Guardrails protected LLM.

In [31]:
import logging

# logging.basicConfig(level=logging.DEBUG)

res = await rails.generate_async(prompt="Hey there!")
print(res)

Hello again! How can I assist you today?
By the way, how are you feeling today?


This is a typical greeting so we have no protective guardrails here. However, we do see the `greeting` flow being activated as the chatbot generates some text from the `bot express greeting` message, and on the next line generates some more text from the `bot ask how are you` message.

Let's try asking a more political question:

In [32]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)

I'm a shopping assistant, I don't like to talk of politics.
If you have any other questions or if there's anything else I can help you with, please let me know.


Here we can see that our `politics` rail is activated and our chatbot dodges the question with `bot answer politics` and follows up with `bot offer help`.

---